In [1]:
# Jupyter version of Dash
from jupyter_dash import JupyterDash

# Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table, Input, Output

# Import MongoClient
from pymongo import MongoClient

# Import Pandas
import pandas as pd

# Change 'animal_shelter' and 'AnimalShelter' to match your CRUD module
from aacCRUD1 import CRUD

# Initialize Dash app
app = JupyterDash(__name__)

# Constants for MongoDB connection
username = "aacUser"
password = "myaacUser"
host = "nv-desktop-services.apporto.com"
port = "31979"
database = "aac"
collection = "animals"

# Data Model
class CRUD:
    def __init__(self, username, password, collection):
        self.username = username
        self.password = password
        self.collection_name = collection
        self.client = MongoClient(f"mongodb://{username}:{password}@{host}:{port}/{database}")
        
    def read(self, query):
        db = self.client.get_database()
        collection = db[self.collection_name]
        data = list(collection.find(query))
        return data

# instance of the CRUD class
db = CRUD(username, password, collection)

# Read data from the database using read method
data = db.read({})

# Convert the data into a pandas DataFrame
df = pd.DataFrame.from_records(data)

# Remove the '_id' column or else the app will crash!
df.drop(columns=['_id'], inplace=True)

# Dash Layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'flex'}),
    html.Center(html.B(html.H1('Austin Animal Center Outcomes Dataset - CS-340 Dashboard - Scott Dixon', style={'color': 'red'}))),
    html.Hr(style={'border-color': 'red'}),
    html.Div(className='buttonRow',
             style={'display': 'flex', 'justify-content': 'center'},
             children=[
                 html.Button(id='submit-button-one', n_clicks=0, children='Find Cats'),
                 html.Button(id='submit-button-two', n_clicks=0, children='Find Dogs')
             ]),
    html.Div(style={'overflow': 'scroll', 'overflow-x': 'auto', 'height': '30px', 'width': 'calc(100% - 60px)'},
             children=[
# data content structure here
             ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=[],
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="multi",
                         row_selectable=False,
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=12
                        ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6', style={'width': 'calc(100% + 300px)', 'margin': '0 auto'})  # Center the map horizontally
])

# Callback to update the datatable and map
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children')],
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('datatable-id', 'selected_rows')]
)
def update_data(button1, button2, index):
    # Determine filter type based on button clicks
    if button1 >= button2:
        filter_type = "Cat"
    else:
        filter_type = "Dog"
    
    # Filter data based on filter type
    filtered_df = df[df["animal_type"] == filter_type]

    # find selected row index
    if index:
        row = index[0]
    else:
        row = 0

    # Geolocation map - need to change size?
    map_children = [
        dl.Map(style={'width': '1000px', 'height': '800px', 'margin': '30 auto'},  # Center the map 
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[filtered_df.iloc[row, 13], filtered_df.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(filtered_df.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(filtered_df.iloc[row, 9])
                                 ])
                             ])
               ])
    ]
    
    # Return filtered data and map
    return filtered_df.to_dict('records'), map_children

if __name__ == '__main__':
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'jupyter_dash'